In [1]:
import pandas as pd
from qdrant_client.models import PointStruct,VectorParams,Distance 
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

d:\Internship\EnterpriseKnowledgeBased\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv(r"D:\Internship\EnterpriseKnowledgeBased\web scrape expirement\FAQ_Data.csv")
df

,question,answers
0,1. What is Offshore Banking Account?,['Account that can be opened by Residents outs...
1,2. Who can open OBU Account?,['Person resident outside Bangladesh including...
2,3. Can customers open a joint OBU account?,"['Yes, customers can open a joint OBU account...."
3,4. What type of account can a customer open?,['A customer can open a Fixed Deposit account ...
4,5. Documents required to open OBU account,"['Document check list for Individuals', 'Copy ..."
...,...,...
94,6. What is the SWIFT code of City Bank PLC?,['Our SWIFT Code is CIBLBDDH. It is also known...
95,"7. If I have IB account in one currency, can I...","['Yes. In this case, cross currency conversion..."
96,1. Is there any tax or any duty applicable on ...,['As per new Offshore Banking Act passed by Na...
97,1. How is City Bank offering such high-interes...,['Offshore banking basically is dependent on f...


In [4]:
#prepare vectors 
model = SentenceTransformer('all-MiniLM-L6-v2')
df['vectors'] = df['answers'].apply(lambda x:model.encode(x))
df

,question,answers,vectors
0,1. What is Offshore Banking Account?,['Account that can be opened by Residents outs...,"[0.05810664, 0.007985746, -0.09814873, -0.0139..."
1,2. Who can open OBU Account?,['Person resident outside Bangladesh including...,"[0.07168972, -0.009594028, -0.069439396, 0.028..."
2,3. Can customers open a joint OBU account?,"['Yes, customers can open a joint OBU account....","[-0.004240756, -0.043203216, -0.0619501, -0.04..."
3,4. What type of account can a customer open?,['A customer can open a Fixed Deposit account ...,"[0.017091805, -0.00031616547, -0.07839836, -0...."
4,5. Documents required to open OBU account,"['Document check list for Individuals', 'Copy ...","[0.025976012, 0.02948665, -0.024018863, -0.058..."
...,...,...,...
94,6. What is the SWIFT code of City Bank PLC?,['Our SWIFT Code is CIBLBDDH. It is also known...,"[-0.05007138, -0.050507728, -0.12777445, -0.04..."
95,"7. If I have IB account in one currency, can I...","['Yes. In this case, cross currency conversion...","[-0.0043450184, 0.014753028, -0.086653866, -0...."
96,1. Is there any tax or any duty applicable on ...,['As per new Offshore Banking Act passed by Na...,"[0.022784665, 0.008492063, -0.042012546, 0.000..."
97,1. How is City Bank offering such high-interes...,['Offshore banking basically is dependent on f...,"[0.07257047, -0.042415928, -0.01589348, -0.019..."


In [6]:
#processing the data 
def prepare_data(row):
    return {
        "id": int(row.name),  # Using the DataFrame's index as the ID
        "payload": {
            "question": row['question'],
            "answers": row['answers']
        },
        "vector": model.encode(row['answers']),  # Encode answers to vectors
    }

# Apply the function to the DataFrame
data_to_insert = df.apply(prepare_data, axis=1).tolist()
data_to_insert

    


[{'id': 0,
  'payload': {'question': '1. What is Offshore Banking Account?',
   'answers': "['Account that can be opened by Residents outside Bangladesh in foreign currency. For example: Non-resident Bangladeshi nationals (NRBs), Persons of Bangladeshi origin (Bangladeshi holding foreign citizenship), Foreign nationals, Companies/Firms registered and operating abroad, Foreign institutional investors etc. can open this OBU account.']"},
  'vector': array([ 5.81066385e-02,  7.98574556e-03, -9.81487334e-02, -1.39865167e-02,
         -4.25510332e-02,  5.14990687e-02, -3.14544067e-02, -6.63828477e-03,
          2.50320765e-03, -7.50530139e-02,  2.52008461e-03, -1.25973925e-01,
         -1.46010285e-02,  4.14211638e-02,  9.80789661e-02,  2.42539290e-02,
         -4.36868370e-02, -5.57931028e-02,  3.60970832e-02, -6.83764145e-02,
         -8.23711883e-03, -2.98828185e-02, -3.50558832e-02, -7.24670216e-02,
         -4.03880291e-02, -1.01469889e-01,  3.23327854e-02, -3.23710963e-02,
          6

In [9]:
#inserting data into qdrant collection 

# Connect to Qdrant
client = QdrantClient(host='localhost', port=6333)

# Choose your collection name
collection_name = 'city_bank_faq_data'

# Create collection if it doesn't exist
if collection_name not in client.get_collections().collections:
    client.create_collection(
        collection_name=collection_name,
        vectors_config={
            'size': len(data_to_insert[0]['vector']),  # Vector dimension
            'distance': 'Cosine'  # Distance metric for vector comparison
        }
    )

# Insert data
for data in data_to_insert:
    client.upsert(
        collection_name=collection_name,
        points=[{
            "id": data['id'],
            "vector": data['vector'],
            "payload": data['payload']
        }]
    )


In [12]:
#search query 
def search_query(query:str):
    vector = model.encode(query)
    search_results = client.search(
        collection_name = collection_name,
        query_vector = vector,
        query_filter=None,
        limit = 5
    )
    return search_results

In [18]:
query = "What is Offshore Banking Account?"
results = search_query(query)
results

[ScoredPoint(id=47, version=47, score=0.65570956, payload={'answers': "['The International Banking (IB) account is an account that can be opened by resident Bangladeshi individual or any Corporate/Firm in foreign currency in Offshore Banking Unit (OBU) of the bank on behalf of someone living abroad (a non-resident). In such cases the account holder will act as a facilitator to their non-resident remitter. The IB account holder and the remitter must have a bonafide relationship.']", 'question': '1. What is City Bank International Banking (IB) Account?'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=97, version=97, score=0.60225344, payload={'answers': "['Offshore banking basically is dependent on foreign borrowings @ 2%-3% margin on SOFR/benchmark rates. City Bank aims to expand its long-term customer deposit base by providing competitive deposit rates aligned with regulatory guidelines. Therefore, City Bank is offering these competitive rates instead of borrowing fro